In [1]:
# Install required libraries
%pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 494.5 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 536.3 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.9/389.9 kB 451.0 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.0/26.0 MB 899.1 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 238.2 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 2.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import required libraries
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from datasets import load_dataset

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Set device to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Download IMDB dataset
imdb_dataset = load_dataset("imdb")

Device: cpu


Generating unsupervised split: 100%|██████████| 50000/50000 [00:00<00:00, 242297.42 examples/s]


In [4]:
imdb_dataset['train'] = imdb_dataset['train'].select(range(10))

imdb_dataset['train']


Dataset({
    features: ['text', 'label'],
    num_rows: 10
})

In [5]:
# Create a tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Create a BERT model
model = BertModel.from_pretrained("bert-base-uncased")

tokenizer, model

(BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
 	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 },
 BertModel(
   (embeddings): BertEmbeddings(
     (word_embeddings): Embedding(30522, 768, padding_idx=0)
     (position_e

In [6]:
# Move model to GPU if available
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [7]:
import numpy as np
# Define a function to generate word embeddings
def generate_word_embeddings(text):
    inputs = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=512,
        return_attention_mask=True,
        return_tensors="pt"
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model(inputs["input_ids"], attention_mask=inputs["attention_mask"])
    embeddings = outputs.last_hidden_state[:, 0, :]
    word_tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"].flatten())
    word_tokens = [word for word in word_tokens if word not in ["[CLS]", "[SEP]", "[PAD]"]]  # Remove special tokens
    return {word: embedding for word, embedding in zip(word_tokens, embeddings.detach().cpu().numpy())}

# Generate word embeddings for the reduced IMDB dataset
embeddings_list = []
for i, review in enumerate(imdb_dataset["train"]["text"]):
    embeddings_dict = generate_word_embeddings(review)
    embeddings_dict["review_id"] = i
    embeddings_list.append(embeddings_dict)

# Convert embeddings to a Pandas DataFrame
embeddings_array = np.array(embeddings_list).squeeze()  # Remove extra dimension
embeddings_df = pd.DataFrame(embeddings_list).set_index("review_id")

# Replace missing values with an empty string
embeddings_df = embeddings_df.fillna("")

# Save the embeddings to a CSV file
embeddings_df.to_csv("imdb_word_embeddings.csv", index=False)

embeddings_df

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


,i,"""",if,this,oh,whoever,when,who
review_id,,,,,,,,
0,"[-0.44797438, -0.17710215, -0.32687828, 0.0088...",,,,,,,
1,,"[-0.05165513, -0.19957957, -0.36138225, 0.1945...",,,,,,
2,,,"[0.15993415, -0.20449525, 0.2851096, -0.370434...",,,,,
3,,,,"[0.31823394, -0.16095433, 0.047124166, -0.0748...",,,,
4,,,,,"[0.10148057, -0.0166932, 0.29143655, 0.0186476...",,,
5,"[0.29642564, -0.09262828, 0.03899841, -0.14921...",,,,,,,
6,,,,,,"[0.15262482, -0.46020463, -0.33316272, -0.1757...",,
7,,,,,,,"[-0.13163166, -0.7564179, -0.47083983, 0.17071...",
8,,,,,,,,"[-0.22594354, 0.05046604, 0.17580913, -0.15455..."
